# Discussion 1: Linear Regression

**TO FILL IN :**


#### About the data: 

#### Purpose: 


## Data Loading

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider
from IPython.display import display, clear_output

# Load the data
file_path = 'data/Hurricane_Irene_Hudson_River.xlsx'
do_data = pd.read_excel('data/Hurricane_Irene_Hudson_River.xlsx', sheet_name = 5).drop(['Piermont D.O. (ppm)'], axis = 1)
rainfall_data = pd.read_excel(file_path, sheet_name='Rainfall').drop(['Piermont  Rainfall Daily Accumulation (Inches)'], axis = 1)
turbidity_data = pd.read_excel(file_path, sheet_name='Turbidity').drop(['Piermont Turbidity in NTU'], axis = 1)



## Data Cleaning

Since we read in our data through multiple excel sheets, we need to merge our different dataframes. The different data sheets all have a column for date, so we can merge our data on this. To better call on the data in our analysis, we can change the column names. Finally, we will update the date to be of datetime type, and set the date as the index. 

In [14]:
# Merge the two datasets

data = rainfall_data.merge(turbidity_data, on = 'Date Time (ET)')
data = data.merge(do_data, on = 'Date Time (ET)')
data.head()

# Update the column names 
data.columns = ['date', 'albany_rainfall', 'norrie_rainfall', 'albany_turbidity', 'norrie_turbidity', 'albany_do', 'norrie_do']

# Make date a datetime object and make it the index

# Convert data to datetime format and set it as index
data['date'] = pd.to_datetime(data['date'])

# Set the date as the index
data.set_index('date', inplace = True)

data.head()



,albany_rainfall,norrie_rainfall,albany_turbidity,norrie_turbidity,albany_do,norrie_do
date,,,,,,
2011-08-25 00:00:00,0.0,0.0,4.0,9.3,7.68,7.81
2011-08-25 00:15:00,0.0,0.0,3.9,8.4,7.60,7.73
2011-08-25 00:30:00,0.0,0.0,4.3,7.9,7.57,7.63
2011-08-25 00:45:00,0.0,0.0,4.7,8.1,7.72,7.67
2011-08-25 01:00:00,0.0,0.0,4.4,8.4,7.74,7.63


## Muliple Linear Regression

Now that our data is cleaned, we can start our multiple linear regression. We will be predicting the turbidity at the Albany sample site, with the rainfall and dissolved oxygen at the Albany sample site as our predictors. Let's check if our model improves at all by adding a second predictor. 

In [15]:
# Define predictors and the target variable
X = data[['albany_rainfall', 'albany_do']]
y = data[['albany_turbidity']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# Create and fit the model
model = LinearRegression().fit(X_train, y_train)
model

# Predict and evaluate
y_pred = model.predict(X_test)
print(mean_squared_error(y_test, y_pred))
#print(f"R-squared: {r2_score(y_test, y_pred)}")


49245.97762215777


## Create a widget to visualize different models

To play around with our model a bit more and visualize different scenarios, we will create a widget using the `ipywidgets` library. 

In [18]:
# Create a widget for selecting predictors
predictor_selector = widgets.SelectMultiple(
    options = data.columns, 
    value = [data.columns[0]],
    description = 'Predictors'

)

# Create a dropdown for selecting the target variable
target_selector = widgets.Dropdown(
    options = data.columns,
    value = data.columns[1],
    description = 'Target'
)

# Button to evaluate the model
evaluate_button = widgets.Button(description = 'Evaluate Model')


# Output widget to display results
output = widgets.Output()

# Define the function to handle button clicks
def evaluate_model(b):
    with output:
        clear_output(wait=True) # Clear output of display area
        
        # Make sure the target is not in the predictors
        selected_predictors = [item for item in predictor_selector.value]
        if target_selector.value in selected_predictors : 
            print("Target variable must not be in the predictors.")
            return
        
        # Prepare the data
        X = data[selected_predictors]
        y = data[target_selector.value]
        
        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
        
        # Create and fit the model
        model = LinearRegression()
        model.fit(X_train, y_train)

        
        # Predict and calculate R^2 and MSE
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        root_mse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        # Display the R^2 score and MSE
        print(f"R^2: {r2:.4f}")
        print(f"MSE: {root_mse:.4f}")




# Display the widgets and connect the button to the function
display(predictor_selector, target_selector, evaluate_button, output)
evaluate_button.on_click(evaluate_model)


SelectMultiple(description='Predictors', index=(0,), options=('albany_rainfall', 'norrie_rainfall', 'albany_tu…

Dropdown(description='Target', index=1, options=('albany_rainfall', 'norrie_rainfall', 'albany_turbidity', 'no…

Button(description='Evaluate Model', style=ButtonStyle())

Output()